In [ ]:
import numpy as np

![Circuit Diagram](image4.png)

The impedance $Z_1$ consists of a resistance of $A \, \Omega$, and an inductance of $B \, \text{mH}$.
The impedance $Z_2$ consists of a resistance of $C \, \Omega$, and a capacitance of $D \, \mu\text{F}$.
The impedance $Z_3$ consists of a resistance of $E \, \Omega$, and an inductance of $F \, \text{mH}$.

The voltage source is $G \angle H^\circ \, \text{V}$ at $J \, \text{Hz}$.


In [1]:
def tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J,freq):   
    X_L_1 = 2*np.pi*freq*B
    X_L_2 = 2*np.pi*freq*F
    X_C = 1/(2*np.pi*freq*D)

    a = ((C*E+X_C*X_L_2)*(C+E) + (D*X_L_2-F*X_C)*(X_L_2-X_C))/((C+E)**2+(X_C-X_L_2)**2) + A
    b = ((D*X_L_2-F*X_C)*(C+E) - (X_L_2-X_C)*(C*E+X_C*X_L_2))/((C+E)**2+(X_C-X_L_2)**2) + X_L_1
    return(a,b)

In [2]:
def curr_I1(A,B,C,D,E,F,G,H,J,freq):
    a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J,freq)
    angle = H*np.pi/180
    c = a*np.cos(angle) + b*np.sin(angle)
    d = a*np.sin(angle) - b*np.cos(angle)
    return (G/(a**2 + b**2))(c,d)

In [3]:
def vol_across_Z2(A,B,C,D,E,F,G,H,J,freq):
    angle = H*np.pi/180
    X_L_1 = 2*np.pi*freq*B
    X_L_2 = 2*np.pi*freq*F
    X_C = 1/(2*np.pi*freq*D)
    c,d = curr_I1(A,B,C,D,E,F,G,H,J,freq)
    e = G*np.cos(angle) + (d*X_L_1-A*c)
    f = -(G*np.sin(angle)+X_L_1*c + A*d)
    return (e,f)

In [1]:
def power_factor(A,B,C,D,E,F,G,H,J,freq):
    c,d = curr_I1(A,B,C,D,E,F,G,H,J,freq)
    return np.cos(H-np.arctan(d/c))

In [3]:
def pow_fac_equal_0(A,B,C,D,E,F,G,H,J,freq):
    a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J,freq)
    angle = H*np.pi/180
    temp = -(np.tan(H)*np.sin(angle)+np.cos(angle))*(a**2 + b**2)/(a*np.cos(angle)*np.tan(H)+b*np.sin(angle)*np.tan(H)+b*np.cos(angle)-a*np.sin(angle))
    if(temp < 0):
        return 1/(2*np.pi*freq*abs(temp))
    else:
        return (2*np.pi*freq*temp)

In [4]:
def phase_ang_diff(A,B,C,D,E,F,G,H,J,freq):
    X_C = 1/(2*np.pi*freq*D)
    e,f = vol_across_Z2(A,B,C,D,E,F,G,H,J,freq)
    # phase angle difference = phase of V3 - phase of I2
    phase = f/C - (f*C + e*X_C)/(e*C - f*X_C)
    return phase
    

In [1]:
def power_loss_percentage(A,B,C,D,E,F,G,H,J,freq):
    X_C = 1/(2*np.pi*freq*D)
    a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J,freq)
    c,d = curr_I1(A,B,C,D,E,F,G,H,J,freq)
    e,f = vol_across_Z2(A,B,C,D,E,F,G,H,J,freq)
    m = (e*C - f*X_C)
    n = (f*C + e*X_C)
    return np.sqrt(m**2+n**2)*C*100/(np.sqrt(c**2+d**2)*a)

In [2]:
def react_power(A,B,C,D,E,F,G,H,J,freq):
    e,f = vol_across_Z2(A,B,C,D,E,F,G,H,J,freq)
    X_C = 1/(2*np.pi*freq*D)
    return (e**2+f**2)/X_C

In [4]:
def energy_consump(A,B,C,D,E,F,G,H,J,freq,t):
    c,d = curr_I1(A,B,C,D,E,F,G,H,J,freq)
    energy = (c**2+d**2)*A*t/(60000)
    return energy

In [5]:
def Vab_and_Vac(A,B,C,D,E,F,G,H,J,freq):
    # Vac - Vab
    c,d = curr_I1(A,B,C,D,E,F,G,H,J,freq)
    X_L_1 = 2*np.pi*freq*B
    return H - np.arctan((X_L_1*c+A*d)/(A*c-X_L_1*d))